In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
import tqdm
import librosa
import librosa.display
import matplotlib.pyplot as plt
# Sklearn 
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report
# TensorFlow
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import numpy as npimage_path_colorbar
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import tensorflow as tf

In [ ]:
from keras.callbacks import ModelCheckpoint
from datetime import datetime

### Reading the 2021 data

In [ ]:
path_xls = "/Users/divya/Desktop/GV_work_dir/data_excel"
xls_2021_name = "2021_2K.xlsx"
xls_2021 = pd.read_excel(os.path.join(path_xls,xls_2021_name))
xls_2021.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'Unique Item ID', 'Title',
       'CallerId', 'Item created date', 'transcription', 'state', 'tags',
       'Recording audio link', 'Format', 'Gender', 'Checksum',
       'Accept/Noisy label', 'file_name'],
      dtype='object')

In [ ]:
xls_2021["file_name"] = xls_2021["Recording audio link"].apply(lambda x:x.split("/")[-1])
xls_2021["file_name"] = xls_2021["file_name"].apply(lambda x:x.split(".")[0])
xls_2021_name = "2021_2K.xlsx"
xls_2021.to_excel(os.path.join(path_xls,xls_2021_name))

### Generating Spectrogram features

#### Function to generate Spectrogram features

In [ ]:
def get_spectrogram_features(x, sr,label_info,_id):
# Extract the audio spectrogram
    try:
        image_path_simple = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2021/{}/simple/".format(label_info)    
        S = librosa.feature.melspectrogram(x, sr=sr, n_mels=128,fmax=8000)
#         plt.figure(figsize=(14,5))
#         librosa.display.specshow(S_dB, x_axis='time',y_axis='mel', sr=sr,fmax=8000)
        # plt.colorbar()
#         plt.savefig(image_path_colorbar+_id,pad_inches = 0,bbox_inches='tight')
        plt.figure(figsize=(8, 5),frameon=False)
        plt.axis("off")
        S_dB = librosa.power_to_db(S, ref=np.max)
        librosa.display.specshow(S_dB, sr=sr,fmax=8000)
        # plt.plot()
        plt.savefig(image_path_simple+_id,pad_inches = 0,bbox_inches='tight')
    except Exception as e:
        print("Error: {}".format(str(e)))

#### Creating Directories to save the Spectrogram images

In [ ]:
noisy_str = "noisy"
image_path_simple = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2021/{}/simple/".format(noisy_str)

if not os.path.exists(image_path_simple):
    os.makedirs(image_path_simple)
    
    
noisy_str = "clean"
image_path_simple = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2021/{}/simple/".format(noisy_str)

if not os.path.exists(image_path_simple):
    os.makedirs(image_path_simple)

# Main

In [ ]:
xls_2021 = xls_2021.iloc[0:10,:].copy() 

In [ ]:
base_audio_path = "/Users/divya/Desktop/GV_work_dir/"
folder_1 = "wave_files_clean_data"
folder_2 = "wave_files_noisy_data"
folder_3 = "wave_files_noisy_data 2"
folder_4 = "wave_files_noisy_data 3"


folder_list = [folder_1, folder_2, folder_3, folder_4]
duration_dict = {}
found_list = []

for _id in tqdm.tqdm(xls_2021["file_name"]):
    for i in folder_list:
        try:
            if "clean" in i:
                label_info = "clean"
            else:
                label_info = "noisy"
                
            image_path_colorbar = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2021/{}/colorbar/".format(label_info)     
            if not os.path.exists(os.path.join(image_path_colorbar,_id+".png")):
                filename_wave = base_audio_path+i+"/"+_id+".wav"
                x,sr = librosa.load(filename_wave, sr=44100)   
                get_spectrogram_features(x, sr,label_info,_id)
                break
            
        except:
            continue

## Previous CNN Model Predictions

In [ ]:
model = keras.models.load_model('./model_noisy.keras')
img_path = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2021/"

In [ ]:
datagen = ImageDataGenerator()
test_Data = datagen.flow_from_directory(img_path,target_size=(224, 224), 
                                        class_mode='binary',batch_size=40,shuffle=False)

Found 838 images belonging to 2 classes.


In [ ]:
import datetime as dt

In [ ]:
start = dt.datetime.now()
Y_pred = model.predict(test_Data,verbose = 1,batch_size = 40)
end = dt.datetime.now()
diff = end-start
diff_sec = diff.total_seconds()
print ("Run time is ", diff_sec)

21/21 [==============================] - 7s 304ms/step
Run time is  10.448875


In [ ]:
y_pred = np.where(Y_pred < 0.5, 1,0)
print(confusion_matrix(test_Data.classes, y_pred))
print(classification_report(test_Data.classes, y_pred))

[[142 212]
 [  2 482]]
              precision    recall  f1-score   support

           0       0.99      0.40      0.57       354
           1       0.69      1.00      0.82       484

    accuracy                           0.74       838
   macro avg       0.84      0.70      0.69       838
weighted avg       0.82      0.74      0.71       838



In [ ]:
y_pred = np.where(Y_pred < 0.15, 1,0)
print(confusion_matrix(test_Data.classes, y_pred))
print(classification_report(test_Data.classes, y_pred))

[[281  73]
 [ 21 463]]
              precision    recall  f1-score   support

           0       0.93      0.79      0.86       354
           1       0.86      0.96      0.91       484

    accuracy                           0.89       838
   macro avg       0.90      0.88      0.88       838
weighted avg       0.89      0.89      0.89       838



## 2020 data for training

In [ ]:
path_xls = "/Users/divya/Desktop/GV_work_dir/data_excel"
xls_2020_name = "20K_files.xlsx"
xls_2020 = pd.read_excel(os.path.join(path_xls,xls_2020_name))

xls_2020["file_name"] = xls_2020["Recording audio link"].apply(lambda x:x.split("/")[-1])
xls_2020["file_name"] = xls_2020["file_name"].apply(lambda x:x.split(".")[0])

In [ ]:
print (xls_2020.columns)
print (xls_2020.shape)

Index(['Unnamed: 0', 'Instance name', 'Unique Item ID', 'Title', 'CallerId',
       'State', 'Block', 'District', 'Item created date', 'transcription',
       'state', 'tags', 'Recording audio link', 'Total listening duration',
       'Item duration', 'published date', 'Format', 'Gender', 'Checksum',
       'Accept/Noisy label', 'file_name'],
      dtype='object')
(20000, 21)


#### Creating Directories to save the Spectrogram images

In [ ]:
noisy_str = "noisy"
image_path_simple = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2020/{}/simple/".format(noisy_str)

if not os.path.exists(image_path_simple):
    os.makedirs(image_path_simple)
    
    
noisy_str = "clean"
image_path_simple = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2020/{}/simple/".format(noisy_str)

if not os.path.exists(image_path_simple):
    os.makedirs(image_path_simple)

# Main

In [ ]:
def get_spectrogram_features(x, sr,label_info,_id,image_path_simple):
# Extract the audio spectrogram
    try:
        S = librosa.feature.melspectrogram(x, sr=sr, n_mels=128,fmax=8000)
        plt.figure(figsize=(8, 5),frameon=False)
        plt.axis("off")
        S_dB = librosa.power_to_db(S, ref=np.max)
        librosa.display.specshow(S_dB, sr=sr,fmax=8000)
        plt.plot()
        plt.savefig(image_path_simple+_id,pad_inches = 0,bbox_inches='tight')
    except Exception as e:
        print("Error: {}".format(str(e)))

In [ ]:
base_audio_path = "/Users/divya/Desktop/GV_work_dir/2020_raw_wave_files/"
folder_1 = "wave_files_clean_data"
folder_2 = "wave_files_noisy_data"

folder_list = [folder_1, folder_2]
duration_dict = {}
found_list = []

for _id in tqdm.tqdm(xls_2020["file_name"]):
    for i in folder_list:
        try:
            if "clean" in i:
                label_info = "clean"
            else:
                label_info = "noisy"
                
            image_path_simple = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2020/{}/simple/".format(label_info)     
            if not os.path.exists(os.path.join(image_path_simple,_id+".png")):
                filename_wave = base_audio_path+i+"/"+_id+".wav"
                x,sr = librosa.load(filename_wave, sr=44100)   
                get_spectrogram_features(x, sr,label_info,_id,image_path_simple)
                break
            
        except:
            continue

## CNN model training

In [ ]:
num_rows = 224
num_columns = 224
num_channels = 3
num_labels = 2

In [ ]:
# Construct model
model_AA_2 = Sequential() # AA - After Augmentation 2- Validation increased
model_AA_2.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, num_channels), activation='relu'))
model_AA_2.add(MaxPooling2D(pool_size=2))
model_AA_2.add(Dropout(0.2))

model_AA_2.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
model_AA_2.add(MaxPooling2D(pool_size=2))
model_AA_2.add(Dropout(0.2))

model_AA_2.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
model_AA_2.add(MaxPooling2D(pool_size=2))
model_AA_2.add(Dropout(0.2))

model_AA_2.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
model_AA_2.add(MaxPooling2D(pool_size=2))
model_AA_2.add(Dropout(0.2))
model_AA_2.add(GlobalAveragePooling2D())

model_AA_2.add(Dense(1, activation='sigmoid'))

# Compile the model
model_AA_2.compile(optimizer='adam',
              loss='binary_crossentropy', # Loss
              metrics=['binary_accuracy'])

# Display model architecture summary
model_AA_2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 223, 223, 16)      208       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 111, 111, 16)      0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 110, 110, 32)      2080      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 55, 55, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 55, 55, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 54, 54, 64)       

### Train and Validation split

In [ ]:
import shutil
test_data = "/Users/divya/Desktop/GV_work_dir/test_data/2021_half.xlsx"
test_data_excel = pd.read_excel(test_data)
test_file_png = [str(x)+".png" for x in test_data_excel["file_name"]]

In [ ]:
import glob
base_path_2021 = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2021"
clean_files_2021 = glob.glob(base_path_2021+'/clean/simple/*.png')
noisy_files_2021 = glob.glob(base_path_2021+'/noisy/simple/*.png')

In [ ]:
clean_files_2021 = [x.split("/")[-1] for x in clean_files_2021]
noisy_files_2021 = [x.split("/")[-1] for x in noisy_files_2021]

In [ ]:
noisy_str = "noisy"
image_path_val_noisy = "/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/val_set/{}/".format(noisy_str)

if not os.path.exists(image_path_val_noisy):
    os.makedirs(image_path_val_noisy)

    
    
noisy_str = "clean"
image_path_val_clean = "Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/val_set/{}/".format(noisy_str)

if not os.path.exists(image_path_val_clean):
    os.makedirs(image_path_val_clean)

In [ ]:
image_path_val_clean

'Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/val_set/clean/'

In [ ]:
#Moving clean files to validation folder
for f in clean_files_2021:
    if f in test_file_png:
        continue
    else:
        shutil.move(base_path_2021+"/clean/simple/"+f, image_path_val_clean)

# Moving noisy files to validation folder
for f in noisy_files_2021:
    if f in test_file_png:
        continue
    else:
        shutil.move(base_path_2021+"/noisy/simple/"+f, image_path_val_noisy)

#### Retraining starts

In [ ]:
datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory('/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2020/',target_size=(224, 224), class_mode='binary',batch_size=40)
val_it = datagen.flow_from_directory('/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/val_set',target_size=(224, 224), class_mode='binary',batch_size=40)
test_it = datagen.flow_from_directory('/Users/divya/Desktop/GV_work_dir/CNN model/spectrogram_data/2021/',target_size=(224, 224), class_mode='binary',batch_size=40)


Found 19991 images belonging to 2 classes.
Found 848 images belonging to 2 classes.
Found 838 images belonging to 2 classes.


In [ ]:
num_epochs = 20
num_batch_size = 40

checkpointer = ModelCheckpoint(filepath='/Users/divya/Desktop/GV_work_dir/CNN model/model_noisy_2.keras',
                               verbose=1, save_best_only=True, monitor = "binary_accuracy")

start = datetime.now()
hist = model_AA_2.fit(train_it,steps_per_epoch=400,epochs = 10, validation_data=val_it,validation_steps= 40,callbacks=[checkpointer], verbose=1)
duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/10
400/400 [==============================] - 500s 1s/step - loss: 0.4140 - binary_accuracy: 0.8207 - val_loss: 0.6122 - val_binary_accuracy: 0.6781

Epoch 00001: binary_accuracy improved from -inf to 0.82071, saving model to /Users/divya/Desktop/GV_work_dir/CNN model/model_noisy_2.keras
Epoch 2/10
400/400 [==============================] - 511s 1s/step - loss: 0.3525 - binary_accuracy: 0.8442

Epoch 00002: binary_accuracy improved from 0.82071 to 0.84416, saving model to /Users/divya/Desktop/GV_work_dir/CNN model/model_noisy_2.keras
Epoch 3/10
400/400 [==============================] - 479s 1s/step - loss: 0.3237 - binary_accuracy: 0.8620

Epoch 00003: binary_accuracy improved from 0.84416 to 0.86200, saving model to /Users/divya/Desktop/GV_work_dir/CNN model/model_noisy_2.keras
Epoch 4/10
400/400 [==============================] - 435s 1s/step - loss: 0.3065 - binary_accuracy: 0.8744

Epoch 00004: binary_accuracy improved from 0.86200 to 0.87443, saving model to /Users/divya/

#### New Model

In [ ]:
Y_pred = model_AA_2.predict(test_it,verbose = 1,batch_size = 40)

21/21 [==============================] - 6s 294ms/step


In [ ]:
y_pred = np.where(Y_pred < 0.5, 1,0)
print(confusion_matrix(test_it.classes, y_pred))
print(classification_report(test_it.classes, y_pred))

[[177 177]
 [256 228]]
              precision    recall  f1-score   support

           0       0.41      0.50      0.45       354
           1       0.56      0.47      0.51       484

    accuracy                           0.48       838
   macro avg       0.49      0.49      0.48       838
weighted avg       0.50      0.48      0.49       838



In [ ]:
model_AA_2.save('./data/model_AA_new.keras')